In [1]:
!pip install google-cloud-documentai pypdf openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 10.2 MB/s eta 0:00:0031m9.8 MB/s eta 0:00:01


In [2]:
import os
from typing import Optional, Literal
from pathlib import Path
from glob import glob

from pydantic import BaseModel, SecretStr
from google.api_core.client_options import ClientOptions
from google.cloud import documentai
from google.oauth2 import service_account
from pypdf import PdfReader
import openai
from IPython.display import display, Markdown

In [9]:
# GCP Config
GCP_PROJECT_ID = "..."
GCP_LOCATION = "us"
GCP_PROCESSOR_ID = "..."
GCP_CREDENTIAL_PATH = "..."

# single file
file_path = "../data/pdf/EasyRAG: Efficient Retrieval-Augmented Generation Framework for Automated Network Operations.pdf"

# multiple files
pdf_path = "../data/pdf"

mime_type = "application/pdf"

os.environ["OPENAI_API_KEY"] = "..."

In [5]:
class GCPConfig(BaseModel):
    GCP_PROJECT_ID: str
    GCP_LOCATION: Literal["us", "eu"]
    GCP_PROCESSOR_ID: str
    GCP_CREDENTIAL_PATH: str

class DocumentAI:
    def __init__(self, filepath, gcp_config: GCPConfig, mime_type="application/pdf"):
        self.filepath = filepath
        self.mime_type = mime_type
        self.client = documentai.DocumentProcessorServiceClient(
            client_options=self._get_client_options(gcp_config), 
            credentials=self._get_credentials(gcp_config)
        )
        self.process_name = self._get_process_name(gcp_config)
        self.load_file()

    def load_file(self):
        with open(self.filepath, "rb") as f:
            content = f.read()

        self.raw_document = documentai.RawDocument(
            content=content, mime_type=self.mime_type
        )
        return self.raw_document

    def run_ocr(self, pages):
        if self.raw_document is None:
            self.load_file()

        process_options = documentai.ProcessOptions(
            individual_page_selector=documentai.ProcessOptions.IndividualPageSelector(
                pages=pages
            )
        )

        request = documentai.ProcessRequest(
            name=self.process_name,
            raw_document=self.raw_document,
            process_options=process_options,
        )

        result = self.client.process_document(request=request)
        return result

    def _get_client_options(self, gcp_config):
        return ClientOptions(
            api_endpoint=f"{gcp_config.GCP_LOCATION}-documentai.googleapis.com"
        )
    
    def _get_credentials(self, gcp_config):
        return service_account.Credentials.from_service_account_file(
            gcp_config.GCP_CREDENTIAL_PATH
        )
    
    def _get_process_name(self, gcp_config):
        return self.client.processor_path(
            gcp_config.GCP_PROJECT_ID,
            gcp_config.GCP_LOCATION,
            gcp_config.GCP_PROCESSOR_ID
        )

class DocumentParser:
    def __init__(self, document):
        self.document = document
        self.text = document.text

    @property
    def pages(self):
        return self.document.pages
        
    def get_content_by_page(self, page_number):
        p = None
        for page in self.pages:
            if page.page_number == page_number:
                p = page
                break
 
        if p is None:
            raise ValueError()

        content = ""

        for line in p.lines:
            for segment in line.layout.text_anchor.text_segments:
                content += self.text[segment.start_index:segment.end_index]
        
        return content

    def get_image_by_page(self, page_number):
        p = None
        for page in self.pages:
            if page.page_number == page_number:
                p = page
                break

        if p is None:
            raise ValueError()

        return p.image.content
    
def refine(ocr_processed, image):
    client = openai.OpenAI()
    
    prompt = f"""# Role
You are an expert at comparing and refining OCR processing and images.
Given an image and OCR processing, refine the OCR processing and answer the question.
If a table exists, convert it to markdown and answer.
Do not include ``` or ```markdown in your answer.
** Don't add anything else and only answer the refined content. **
# ocr_processed
{ocr_processed}
# image
"""
    messages = [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": prompt},
                {"type": "image_url", "image_url": {"url": f"data:image/png;base64, {image}"}}
            ]
        }
    ]

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
    )
    response_text = response.choices[0].message.content
    return response_text

def transform_to_base64(byte):
    import base64
    return base64.b64encode(byte).decode("utf-8")

In [10]:
gcp_config = GCPConfig(
    GCP_PROJECT_ID=GCP_PROJECT_ID,
    GCP_LOCATION=GCP_LOCATION,
    GCP_PROCESSOR_ID=GCP_PROCESSOR_ID,
    GCP_CREDENTIAL_PATH=GCP_CREDENTIAL_PATH
)
dai = DocumentAI(file_path, gcp_config)

pages = len(PdfReader(file_path).pages)
page_splits = [range(i, min(i+14, pages+1)) for i in range(1, pages+1, 15)]

for page_split in page_splits:
    result = dai.run_ocr(page_split)
    document = result.document

    dp = DocumentParser(document)
    
    for page in page_split:
        content = dp.get_content_by_page(page)
        image = dp.get_image_by_page(page)
        image = transform_to_base64(image)

        print(f"{page=}")
        display(Markdown(refine(content, image)))

page=1


arXiv:2410.10315v2 [cs.CL] 15 Oct 2024  
EasyRAG: Efficient Retrieval-Augmented Generation Framework for Automated Network Operations  
Zhangchi Feng¹, Dongdong Kuang¹, Zhongyuan Wang¹, Zhijie Nie¹, Yaowei Zheng¹, Richong Zhang¹*  
¹CCSE, School of Computer Science and Engineering, Beihang University, Beijing, China  
{zcmuller, kuangdd, wangzy23, hiyouga}@buaa.edu.cn, {niezj, zhangrc}@act.buaa.edu.cn  

Abstract  
This paper presents EasyRAG, a simple, lightweight, and efficient retrieval-augmented generation framework for automated network operations. Our framework has three advantages. The first is accurate question answering. We designed a straightforward RAG scheme based on (1) a specific data processing workflow (2) dual-route sparse retrieval for coarse ranking (3) LLM Reranker for reranking (4) LLM answer generation and optimization. This approach achieved first place in the GLM4 track in the preliminary round and second place in the GLM4 track in the semifinals. The second is simple deployment. Our method primarily consists of BM25 retrieval and BGE-reranker reranking, requiring no fine-tuning of any models, occupying minimal VRAM, easy to deploy, and highly scalable; we provide a flexible code library with various search and generation strategies, facilitating custom process implementation. The last one is efficient inference. We designed an efficient inference acceleration scheme for the entire coarse ranking, reranking, and generation process that significantly reduces the inference latency of RAG while maintaining a good level of accuracy; each acceleration scheme can be plug-and-play into any component of the RAG process, consistently enhancing the efficiency of the RAG system. Our code and data are released at https://github.com/BUAADreamer/EasyRAG.

1 Introduction  
Our solution can be summarized by Fig. 1, which includes a data processing workflow (Section 1.1) and the RAG process (Section 1.2).  
*Corresponding author  
'This work is a technical report of our solution at the 2024 (7th) CCF International AIOps Challenge. The official website of the Challenge is https://competition.aiops-challenge.com

1.1 Ingestion  
1.1.1 zedx file processing  
Due to the discovery that the original processing script missed some files, we have reprocessed the zedx files using the following steps:  
1. zedx Decompression: Decompress the official source data from four .zedx files, obtaining four packages of HTML documents.  
2. Path Parsing: Read the knowledge path and the actual file path from the nodetree.xml in each document package.  
3. Document Extraction: Extract the text, image titles, and image paths from each HTML document using BeautifulSoup.  
4. Saving: Save the document text in txt format, maintaining the relative location consistent with the HTML document. Also, save the knowledge path, file path, and image path information.  

1.1.2 Text Segmentation  
Segmentation Settings: We used SentenceSplitter for document segmentation, initially splitting into sentences using Chinese punctuation, then merging according to the set text block size. The used block size (chunk-size) is 1024, and the block overlap size (chunk-overlap) is 200.  

Eliminating Path Influence in Segmentation: In practice, we found that the original implementation of llama-index used a simple but unstable method of handling path information, subtracting the file path length from the text length to determine the actual text length used. This approach could cause different segmentation results with the same chunk-size and chunk-overlap, depending on the data path. During the preliminary competition, we observed that changing paths could lead to a fluctuation of...

page=2


1. Ingestion
   - Documents

2. EasyRAG
   - question
   - bm25
   - query rewrite
   - split
   - chunks:
     - chunk 1
     - chunk 2
   - chunk retrieval
   - metadata:
     - file path
   - Rerank Doc Embed / Retrieval filter
   - knowledge path --> Retrieval Doc Embed
   - extract:
     - LLM Context Embed
     - image content
     - generate describe
     - paddleocr
     - filtered images
     - image path
     - word rule
     - sentence rule
     - glm4v-96-chat
     - filter
   - chunk 1
   - chunk n-1
   - chunk k
   - chunk n
   - bge- -> reranker-v2-
   - minicom
   - rerank
   - generate
   - GLM-4
   - Ans':
   - path retrieval
   - chunk ntm
   - query <--
   - refine
     - GLM-4
     - Ans

Figure 1: EasyRAG Framework

Up to 3 percentage points in the final evaluation results is unacceptable in practice. To address this issue, we implemented a custom segmentation class that eliminates the use of path length, ensuring stable reproducibility.

1.1.3 Image Information Extraction
- Image Content Extraction Using a Multimodal Large Model
  - We extracted information from all images using GLM-4V-9B (GLM et al., 2024). The simple prompt that achieves good results is: 
    - Briefly describe the image.
  
- Image Filtering Based on Various Rules
  - We found that a small number of images are beneficial for the final question answering, but not all images are useful. Therefore, we designed a flexible strategy to filter out useless images using the following steps:
    1. Use the PP-OCRv4 model to extract text content from images and filter out images that do not contain Chinese.
    2. Filter images whose titles contain specific keywords (e.g., network diagrams, architecture).
    3. Filter images that are referenced in the text in a specific way (e.g., configuration as shown in Figure x, file as shown in Figure x).
  
With these filtering steps, we reduced the number of images from an original 6000 to fewer than 200. The filtering process is easily configurable, allowing for tuning to suit real-world scenarios.

1.2 RAG Pipeline
1.2.1 Query Rewriting
- Given the queries were very brief, we identified issues with some queries being semantically awkward or having unclear keywords. For instance, "What types of alarms are there in EMSPLUS?" and "What are the sources of faults?". Before inputting these queries into the RAG Pipeline, we used a Large Language Model (LLM, GLM4) for query rewriting, which involved two methods: query expansion and Hypothetical Document Embedding (HyDE) (Gao et al., 2022).

Query Expansion
- During the preliminary round, we summarized the characteristics of queries in the current operational maintenance scenario:
  - Technical keywords in queries are crucial.
  - Queries are short and vary greatly in the amount of information provided.

In this context, we attempted to summarize the key terms in the queries or other potentially relevant keywords using the LLM, i.e., using the LLM's knowledge for keyword association and summary in the fields of operation and communication. This is referred to as keyword expansion.

page=3


...

Method | Time(s) | Similarity (%) | Rank
--- | --- | --- | ---
8-layer | 1.67 | 73 | 2.5
12-layer | 2.20 | 88 | 3.2
20-layer | 3.58 | 86 | 4.0
28-layer | 5.25 | 100 | 6.0
40-layer | 7.71 | 100 | 5.4
Maximum (0.1) | 2.59 | 90 | 3.7
Maximum (0.2) | 3.55 | 96 | 4.5
Maximum (0.4) | 4.57 | 97 | 5.4
Entropy (0.2) | 2.74 | 89 | 3.4
Entropy (0.4) | 3.37 | 91 | 3.6
Entropy (0.6) | 4.01 | 91 | 4.0

Compression Algorithm | Compression Rate (%) | Tokens Saved | Accuracy | Time (s)
--- | --- | --- | --- | ---
Original Content | 100 | 0 | 94.49 | 9.30
LLMLingua(0.5) | 62.80 | 143k | 83.44 | 10.47
LongLLMLingua(0.5) | 62.80 | 143k | 80.86 | 10.52
BM25-Extract(0.5) | 65.92 | 160k | 86.48 | 8.16
BM25-Extract(0.8) | 83.84 | 59k | 89.00 | 7.70

page=11


Weijie Liu, Peng Zhou, Zhiruo Wang, Zhe Zhao, Haotang Deng, and Qi Ju. 2020. FastBERT: a self-distilling BERT with adaptive inference time. In Proceedings of the 58th Annual Meeting of the Association for Computational Linguistics, pages 6035-6044, Online. Association for Computational Linguistics.  
Xing Han Lù. 2024. Bm25s: Orders of magnitude faster lexical search via eager sparse scoring. Preprint, arXiv:2407.03618.  
Sheila Teo. 2023. How I won Singapore's GPT-4 prompt engineering competition.  
Liang Wang, Nan Yang, and Furu Wei. 2023. Query2doc: Query expansion with large language models. arXiv preprint arXiv:2303.07678.  

## A Question-and-Answer Prompt Templates  

### A.1 Markdown Format Question-and-Answer Template  
## Objective  
Please, based on the information from k private domain documents about 5G operational maintenance, answer the given question.  
## Requirements  
1. You may itemize your answer; be as detailed and specific as possible.  
2. Do not merely repeat information from the context.  
3. Do not use your own knowledge; rely solely on the content from the context documents.  
## Context  
{context_str}  
## Question  
{query_str}  
## Answer  
Jason Wei, Xuezhi Wang, Dale Schuurmans, Maarten Bosma, Fei Xia, Ed Chi, Quoc V Le, Denny Zhou, et al. 2022. Chain-of-thought prompting elicits reasoning in large language models. Advances in neural information processing systems, 35:24824-24837.  

### A.2 Chain of Thought Question-and-Answer Template  
Context information as follows:  
{context_str}  
Ji Xin, Raphael Tang, Jaejun Lee, Yaoliang Yu, and Jimmy Lin. 2020. DeeBERT: Dynamic early exiting for accelerating BERT inference. In Proceedings of the 58th Annual Meeting of the Association for Computational Linguistics, pages 2246-2251, Online. Association for Computational Linguistics.  
Please answer the following question based on the context information rather than your own knowledge. Think step by step, first provide an analysis process, then generate an answer:  
{query_str}  
Answer:  
11

page=12


A.3 Focused Question-and-Answer Template  
Context information as follows:  
{context_str}  
Please answer the following question based on the context information rather than your own knowledge. You may itemize your answer. Document 0's content is particularly important, consider it carefully. If the context does not contain relevant knowledge, you may respond with 'uncertain'. Do not simply restate the context information:  
{query_str}  
Answer:  

B Answer Integration Template  
Context:  
{top1_content_str}  
You will see a question and a corresponding reference answer. Please, based on the context knowledge and not your own knowledge, supplement the reference answer to make it more complete in addressing the question. Please note, strictly retain every character of the reference answer and reasonably integrate your supplement with the reference answer to produce a longer, more complete answer containing more terms and itemization.  
Question:  
{query_str}  
Reference answer:  
{answer_str}  
New answer:  
12